In [1]:
import pandas as pd
import numpy as np


In [2]:
Date='2023-03-10'
Line='3-10'
Model='LONGSP'
Startdate='2022-12-01'
finishDate='2023-03-09'


In [3]:
import pyodbc
import pandas as pd
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '192.168.101.219' 
database = 'DataforAnalysis' 
username = 'DATALYZER' 
password = 'NMB54321'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# select 26 rows from SQL table to insert in dataframe.
query = "SELECT [Projection1],[Datum_probe],[Max_force],[Set_Dim_A],[Set_Dim_B],[Set_Dim_C],[Pivot_Height]FROM [DataforAnalysis].[dbo].[DataML] WHERE [Model] = 'LONGSP' AND [Line] = '"+Line+"' AND [Date] BETWEEN '"+Startdate+"' and '"+finishDate+"'and ([Projection1] is not null and [Datum_probe] is not null and [Max_force] is not null and [Set_Dim_A] is not null and [Set_Dim_B] is not null and [Set_Dim_C] is not null and [Pivot_Height] is not null)and [Projection1] !='0' and [Max_force] >=100 and [Set_Dim_A] >=0.3775 and [Set_Dim_B] >=0.3775 and [Set_Dim_C] >=0.3775 and [Max_force] <= 3700.00 and [Set_Dim_A] <= 0.6537 and [Set_Dim_B] <= 0.6537 and  [Set_Dim_C] <= 0.6537;"
df = pd.read_sql(query, cnxn)
print(df.head(100000))


C:\Users\IT\AppData\Local\Temp\ipykernel_3676\2211626856.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)


       Projection1  Datum_probe Max_force  Set_Dim_A  Set_Dim_B  Set_Dim_C  \
0           0.6048   358.459991      1180     0.5599     0.5171     0.5613   
1           0.6076   358.459991      1180     0.5488     0.5283     0.5662   
2           0.6082   358.459991      1180     0.5522     0.5205     0.5652   
3           0.6167   358.459991      1180     0.5593     0.5139     0.5610   
4           0.6138   358.459991      1180     0.5604     0.5126     0.5603   
...            ...          ...       ...        ...        ...        ...   
99995       0.5189   455.570007       965     0.5194     0.5008     0.5154   
99996       0.5077   482.540009      1041     0.5163     0.4947     0.5057   
99997       0.5077   460.329987       510     0.5200     0.5068     0.5092   
99998       0.5251   453.059998       978     0.5130     0.5059     0.5154   
99999       0.5000   478.700012       919     0.5176     0.4911     0.5033   

       Pivot_Height  
0            -9.414  
1            -9.416

In [4]:
datasets = df


In [5]:
datasets['Projection1']=pd.cut(datasets['Projection1'],bins=[-np.inf,0.4647,0.5664,np.inf],labels=['fail_low','pass','fail_high'])
datasets['Projection1'].replace('fail_low','fail',inplace=True)
datasets['Projection1'].replace('fail_high','fail',inplace=True)


In [6]:
datasets


,Projection1,Datum_probe,Max_force,Set_Dim_A,Set_Dim_B,Set_Dim_C,Pivot_Height
0,fail,358.459991,1180,0.5599,0.5171,0.5613,-9.414
1,fail,358.459991,1180,0.5488,0.5283,0.5662,-9.416
2,fail,358.459991,1180,0.5522,0.5205,0.5652,-9.414
3,fail,358.459991,1180,0.5593,0.5139,0.5610,-9.415
4,fail,358.459991,1180,0.5604,0.5126,0.5603,-9.416
...,...,...,...,...,...,...,...
529284,pass,470.029999,799,0.5161,0.5006,0.5191,-9.383
529285,pass,464.480011,1632,0.5132,0.5093,0.5224,-9.387
529286,pass,481.809998,1360,0.5203,0.5036,0.5241,-9.387
529287,pass,465.089996,1774,0.5162,0.5051,0.5195,-9.384


In [7]:
from collections import Counter
print(Counter(datasets['Projection1']))


Counter({'pass': 526025, 'fail': 3264})


In [8]:
from imblearn.over_sampling import SMOTE
# over_sampling คือการทำค่า fail 1284 ให้เท่ากับค่า pass 271k เพื่อให้ data ในการ train เพิ่มมากขึ้น
from collections import Counter

k = 3
X = datasets.loc[:, datasets.columns != 'Projection1']
y = datasets.Projection1

# sampling_strategy=0.1 oversampling 10%
sm = SMOTE(sampling_strategy='minority', k_neighbors=k, random_state=100)
X_res, y_res = sm.fit_resample(X, y)

datasets = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)


In [9]:
from collections import Counter
print(Counter(datasets['Projection1']))


Counter({'fail': 526025, 'pass': 526025})


In [10]:
datasets


,Datum_probe,Max_force,Set_Dim_A,Set_Dim_B,Set_Dim_C,Pivot_Height,Projection1
0,358.459991,1180.0,0.559900,0.517100,0.561300,-9.414000,fail
1,358.459991,1180.0,0.548800,0.528300,0.566200,-9.416000,fail
2,358.459991,1180.0,0.552200,0.520500,0.565200,-9.414000,fail
3,358.459991,1180.0,0.559300,0.513900,0.561000,-9.415000,fail
4,358.459991,1180.0,0.560400,0.512600,0.560300,-9.416000,fail
...,...,...,...,...,...,...,...
1052045,354.609985,678.0,0.556378,0.548234,0.562045,-9.416167,fail
1052046,467.470547,1075.455548,0.509470,0.504512,0.533591,-9.384811,fail
1052047,470.685325,691.937929,0.513550,0.465923,0.518588,-9.381938,fail
1052048,459.959991,1253.0,0.519935,0.484235,0.524405,-9.396332,fail


In [11]:
Datum_3LCL = 124.99
Datum_2LCL = 249.99
Datum_LCL = 374.99
Datum_CL = 499.99
Datum_UCL = 624.99
Datum_2UCL = 749.99
Datum_3UCL = 875

Max_3LCL = 299.99
Max_2LCL = 833.32
Max_LCL = 1366.66
Max_CL = 1899.99
Max_UCL = 2433.32
Max_2UCL = 2966.66
Max_3UCL = 3500

Set_Dim_A_3LCL = 0.4774
Set_Dim_A_2LCL = 0.4901
Set_Dim_A_LCL = 0.5028
Set_Dim_A_CL = 0.5155
Set_Dim_A_UCL = 0.5282
Set_Dim_A_2UCL = 0.5409
Set_Dim_A_3UCL = 0.5537


Set_Dim_B_3LCL = 0.4774
Set_Dim_B_2LCL = 0.4901
Set_Dim_B_LCL = 0.5058
Set_Dim_B_CL = 0.5155
Set_Dim_B_UCL = 0.5282
Set_Dim_B_2UCL =0.5409
Set_Dim_B_3UCL = 0.5537

Set_Dim_C_3LCL = 0.4774
Set_Dim_C_2LCL = 0.4901
Set_Dim_C_LCL = 0.5028
Set_Dim_C_CL = 0.5155
Set_Dim_C_UCL = 0.5282
Set_Dim_C_2UCL =0.5409
Set_Dim_C_3UCL = 0.5537

Pivot_Height_3LCL = -9.4030
Pivot_Height_2LCL = -9.3945
Pivot_Height_LCL = -9.3860
Pivot_Height_CL = -9.3776
Pivot_Height_UCL = -9.3691
Pivot_Height_2UCL = -9.3606
Pivot_Height_3UCL = -9.3523


datasets['Datum_probe']=pd.cut(datasets['Datum_probe'],bins=[-np.inf, Datum_3LCL, Datum_2LCL, Datum_LCL, Datum_CL, Datum_UCL, Datum_2UCL, Datum_3UCL, np.inf],labels=['A','B','C','D','E','F','G','H'])

datasets['Max_force']=pd.cut(datasets['Max_force'],bins=[-np.inf, Max_3LCL, Max_2LCL, Max_LCL, Max_CL, Max_UCL, Max_2UCL, Max_3UCL, np.inf],labels=['A','B','C','D','E','F','G','H'])

datasets['Set_Dim_A']=pd.cut(datasets['Set_Dim_A'],bins=[-np.inf, Set_Dim_A_3LCL, Set_Dim_A_2LCL, Set_Dim_A_LCL, Set_Dim_A_CL, Set_Dim_A_UCL, Set_Dim_A_2UCL, Set_Dim_A_3UCL,np.inf],labels=['A','B','C','D','E','F','G','H'])

datasets['Set_Dim_B']=pd.cut(datasets['Set_Dim_B'],bins=[-np.inf, Set_Dim_B_3LCL, Set_Dim_B_2LCL, Set_Dim_B_LCL, Set_Dim_B_CL, Set_Dim_B_UCL, Set_Dim_B_2UCL, Set_Dim_B_3UCL, np.inf],labels=['A','B','C','D','E','F','G','H'])

datasets['Set_Dim_C']=pd.cut(datasets['Set_Dim_C'],bins=[-np.inf, Set_Dim_C_3LCL, Set_Dim_C_2LCL, Set_Dim_C_LCL, Set_Dim_C_CL, Set_Dim_C_UCL, Set_Dim_C_2UCL, Set_Dim_C_3UCL, np.inf],labels=['A','B','C','D','E','F','G','H'])

datasets['Pivot_Height']=pd.cut(datasets['Pivot_Height'],bins=[-np.inf, Pivot_Height_3LCL, Pivot_Height_2LCL, Pivot_Height_LCL, Pivot_Height_CL, Pivot_Height_UCL, Pivot_Height_2UCL, Pivot_Height_3UCL,np.inf],labels=['A','B','C','D','E','F','G','H'])



In [12]:
datasets


,Datum_probe,Max_force,Set_Dim_A,Set_Dim_B,Set_Dim_C,Pivot_Height,Projection1
0,C,C,H,E,H,A,fail
1,C,C,G,F,H,A,fail
2,C,C,G,E,H,A,fail
3,C,C,H,D,H,A,fail
4,C,C,H,D,H,A,fail
...,...,...,...,...,...,...,...
1052045,C,B,H,G,H,A,fail
1052046,D,C,D,C,F,D,fail
1052047,D,B,D,A,E,D,fail
1052048,D,C,E,B,E,B,fail


In [13]:
from pyarc import CBA, TransactionDB
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report

train, test = train_test_split(datasets, test_size=0.1)

txns_train = TransactionDB.from_DataFrame(train)
txns_test = TransactionDB.from_DataFrame(test)

cba = CBA (support = 0.09, confidence = 0.8, algorithm ='m1')
cba.fit(txns_train)

y_pred = cba.predict(txns_test)
y_test = test['Projection1']
print ((classification_report(y_test,y_pred)))



              precision    recall  f1-score   support

        fail       0.94      0.37      0.53     52811
        pass       0.61      0.98      0.75     52394

    accuracy                           0.67    105205
   macro avg       0.77      0.67      0.64    105205
weighted avg       0.77      0.67      0.64    105205



In [14]:
datasp =(classification_report(y_test,y_pred))


In [15]:
da=datasp.split("\n")


In [16]:
from pandas import DataFrame

data1 ={
'Date':Date,
'Line':Line,
'Model':Model,
'Details':[(datasp[63:67]),(datasp[117:121]),(datasp[168:176]),(datasp[221:230]),(datasp[272:284])
],
'Procision':[datasp[74:78],datasp[128:132],(''),datasp[128:132],datasp[237:241]
],
'recall':[datasp[84:88],datasp[138:142],(''),datasp[247:251],datasp[301:305]
],
'f1_score':[datasp[94:98],datasp[148:152],(datasp[203:207]),datasp[257:261],datasp[311:315]
],

'support':[datasp[103:108],datasp[157:162],datasp[209:217],datasp[265:271],datasp[319:325]
],
}
dff = pd.DataFrame(data1,columns=['Date','Line','Model','Details','Procision','recall','f1_score','support'])
print("displaying the dataframe with data as record")
print(dff)
print("-------------------------------Converting into csv file with a new a new file name ------------------------------------")
# df.to_csv("C:\\Users\\IT\\Desktop\\Date ML\\Accuracy ML\\LSP3-14 Accuracy.csv")
# df=pd.read_csv("C:\\Users\\IT\\Desktop\\Date ML\\Accuracy ML\\LSP3-14 Accuracy.csv")
# print(df)


displaying the dataframe with data as record
         Date  Line   Model       Details Procision recall f1_score   support
0  2023-03-10  3-10  LONGSP          fail      0.94   0.37     0.53     52811
1  2023-03-10  3-10  LONGSP          pass      0.61   0.98     0.75     52394
2  2023-03-10  3-10  LONGSP      accuracy                      0.67    105205
3  2023-03-10  3-10  LONGSP     macro avg      0.61   0.67     0.64    105205
4  2023-03-10  3-10  LONGSP  weighted avg      0.77   0.67     0.64    105205
-------------------------------Converting into csv file with a new a new file name ------------------------------------


In [17]:
dff


,Date,Line,Model,Details,Procision,recall,f1_score,support
0,2023-03-10,3-10,LONGSP,fail,0.94,0.37,0.53,52811
1,2023-03-10,3-10,LONGSP,pass,0.61,0.98,0.75,52394
2,2023-03-10,3-10,LONGSP,accuracy,,,0.67,105205
3,2023-03-10,3-10,LONGSP,macro avg,0.61,0.67,0.64,105205
4,2023-03-10,3-10,LONGSP,weighted avg,0.77,0.67,0.64,105205


In [18]:
import pyodbc
import pandas as pd
# insert data from csv file into dataframe.
# working directory for csv file: type "pwd" in Azure Data Studio or Linux
# working directory in Windows c:\users\username
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '192.168.101.219' 
database = 'DataforAnalysis' 
username = 'DATALYZER' 
password = 'NMB54321'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
for index, row in dff.iterrows():

     cursor.execute("INSERT INTO [DataforAnalysis].[dbo].[accuracyDataML] (Date,Line,Model,Details,Procision,recall,f1_score,support) values ('"+row.Date+"','"+row.Line+"','"+row.Model+"','"+row.Details+"','"+row.Procision+"','"+row.recall+"','"+row.f1_score+"','"+row.support+"')")
     # ,(row.Date,row.betweenDate,row.Model,)Procision
                    # ('"+row.Date+"','"+row.betweenDate+"','"+row.Model+"','"+row.Line+"','"+row.Datum_probe+"','"+ a +"','"+row.Set_Dim_A+"','"+row.Set_Dim_B+"','"+row.Set_Dim_C+"','"+row.Pivot_Height+"','"+row.Projection+"','"+row.Support+"','"+row.Confidence+"')")
cnxn.commit()
cursor.close()


In [19]:
cba1 =cba.clf.rules


In [20]:
aa =" "
dd=[]
for  x in range(len(cba1)):
    aa += str (cba1[x]).replace("{","").replace("}","").replace("Projection1=","")
    dd=aa.split("CAR")
print(dd)


[' ', ' Set_Dim_B=C,Set_Dim_C=F => fail sup: 0.14 conf: 0.94 len: 3, id: 10', ' Datum_probe=D,Max_force=B,Set_Dim_C=D => pass sup: 0.09 conf: 0.94 len: 4, id: 15', ' Max_force=B,Set_Dim_C=D => pass sup: 0.09 conf: 0.94 len: 3, id: 16', ' Max_force=C,Pivot_Height=D,Set_Dim_C=F => fail sup: 0.09 conf: 0.94 len: 4, id: 2', ' Max_force=C,Set_Dim_C=F => fail sup: 0.13 conf: 0.94 len: 3, id: 4']


In [21]:
fid=[]
fid01=[]
Sup=[]
Conf=[]
for op in range(len(dd)):
    dat=(dd[op].split())
    fid+=(dat[0:1])
    fid01+=dat[2:3]
    Sup+=dat[4:5]
    Conf+=dat[6:7]



In [22]:
my_array=[]
ok =[]
i=0
while i < len(fid):
    my_array+= fid[i].split()
    ok +=[my_array[i]+','+'Null=Null'+','+'Null=Null'+','+'Null=Null'+','+'Null=Null']
    i=i+1


In [23]:
Sp=[]
KPIV1=[]
KPIV2=[]
KPIV3=[]
KPIV4=[]
for i in ok:
    Sp1=i.split(',')
    KPIV1+=[Sp1[0]]
    KPIV2+=[Sp1[1]]
    KPIV3+=[Sp1[2]]
    KPIV4+=[Sp1[3]]

KPIVOK=[KPIV1],[KPIV2],[KPIV3],[KPIV4]


In [24]:
dataok=[]
for i in range(len(KPIVOK)):
    dataok+=(KPIVOK[i])


In [25]:
sp1=[]
for i in (dataok[0]):
    sp1+=[(i.split('='))]

sp2=[]
for i in (dataok[1]):
    sp2+=[(i.split('='))]

sp3=[]
for i in (dataok[2]):
    sp3+=[(i.split('='))]

sp4=[]
for i in (dataok[3]):
    sp4+=[(i.split('='))]


In [26]:
Cl1=[]
for i in sp1:
    Cl1+=([i[0]])

Cl2=[]
for i in sp1:
    Cl2+=([i[1]])

Cl3=[]
for i in sp2:
    Cl3+=([i[0]])

Cl4=[]
for i in sp2:
    Cl4+=([i[1]])

Cl5=[]
for i in sp3:
    Cl5+=([i[0]])

Cl6=[]
for i in sp3:
    Cl6+=([i[1]])


In [27]:
Cl5


['Null', 'Set_Dim_C', 'Null', 'Set_Dim_C', 'Null']

In [28]:
Cl5


['Null', 'Set_Dim_C', 'Null', 'Set_Dim_C', 'Null']

In [29]:
betweenDate = Startdate+' - '+finishDate


In [30]:
indexok=[]
for i in range(len(my_array)):
    indexok+=([i+1])


In [31]:
import pandas as pd
data ={
'Date':Date,
'betweenDate':betweenDate,
'Model':Model,
'Line':Line,
'Rangeindex':indexok,
'Cl1':Cl1,
'Cl2':Cl2,
'Cl3':Cl3,
'Cl4':Cl4,
'Cl5':Cl5,
'Cl6':Cl6,
'Projection':fid01,
'Support':Sup,
'Confidence':Conf,}
df = pd.DataFrame(data)

df.loc[df['Cl1'] == 'Datum_probe', 'Datum_probe'] = df['Cl2']
df.loc[df['Cl3'] == 'Datum_probe', 'Datum_probe'] = df['Cl4']
df.loc[df['Cl5'] == 'Datum_probe', 'Datum_probe'] = df['Cl6']

df.loc[df['Cl1'] == 'Max_force', 'Max_force'] = df['Cl2']
df.loc[df['Cl3'] == 'Max_force', 'Max_force'] = df['Cl4']
df.loc[df['Cl5'] == 'Max_force', 'Max_force'] = df['Cl6']

df.loc[df['Cl1'] == 'Set_Dim_A', 'Set_Dim_A'] = df['Cl2']
df.loc[df['Cl3'] == 'Set_Dim_A', 'Set_Dim_A'] = df['Cl4']
df.loc[df['Cl5'] == 'Set_Dim_A', 'Set_Dim_A'] = df['Cl6']

df.loc[df['Cl1'] == 'Set_Dim_B', 'Set_Dim_B'] = df['Cl2']
df.loc[df['Cl3'] == 'Set_Dim_B', 'Set_Dim_B'] = df['Cl4']
df.loc[df['Cl5'] == 'Set_Dim_B', 'Set_Dim_B'] = df['Cl6']

df.loc[df['Cl1'] == 'Set_Dim_C', 'Set_Dim_C'] = df['Cl2']
df.loc[df['Cl3'] == 'Set_Dim_C', 'Set_Dim_C'] = df['Cl4']
df.loc[df['Cl5'] == 'Set_Dim_C', 'Set_Dim_C'] = df['Cl6']

df.loc[df['Cl1'] == 'Pivot_Height', 'Pivot_Height'] = df['Cl2']
df.loc[df['Cl3'] == 'Pivot_Height', 'Pivot_Height'] = df['Cl4']
df.loc[df['Cl5'] == 'Pivot_Height', 'Pivot_Height'] = df['Cl6']

df.index = df.index.astype(str)
df = pd.DataFrame(df,columns=['Date','betweenDate','Model','Line','Rangeindex','Datum_probe','Max_force','Set_Dim_A','Set_Dim_B','Set_Dim_C','Pivot_Height','Projection','Support','Confidence'])
print("displaying the dataframe with data as record")
print(df)
print("-------------------------------Converting into csv file with a new a new file name ------------------------------------")
# df.to_csv("C:\\Users\\IT\\Desktop\\Date ML\\3-14 KPIV.csv")
# df=pd.read_csv("C:\\Users\\IT\\Desktop\\Date ML\\3-14 KPIV.csv")
# print(df)



displaying the dataframe with data as record
         Date              betweenDate   Model  Line  Rangeindex Datum_probe  \
0  2023-03-10  2022-12-01 - 2023-03-09  LONGSP  3-10           1         NaN   
1  2023-03-10  2022-12-01 - 2023-03-09  LONGSP  3-10           2           D   
2  2023-03-10  2022-12-01 - 2023-03-09  LONGSP  3-10           3         NaN   
3  2023-03-10  2022-12-01 - 2023-03-09  LONGSP  3-10           4         NaN   
4  2023-03-10  2022-12-01 - 2023-03-09  LONGSP  3-10           5         NaN   

  Max_force Set_Dim_A Set_Dim_B Set_Dim_C Pivot_Height Projection Support  \
0       NaN       NaN         C         F          NaN       fail    0.14   
1         B       NaN       NaN         D          NaN       pass    0.09   
2         B       NaN       NaN         D          NaN       pass    0.09   
3         C       NaN       NaN         F            D       fail    0.09   
4         C       NaN       NaN         F          NaN       fail    0.13   

  Confidenc

In [32]:
import pyodbc
import pandas as pd
# insert data from csv file into dataframe.
# working directory for csv file: type "pwd" in Azure Data Studio or Linux
# working directory in Windows c:\users\username
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '192.168.101.219' 
database = 'DataforAnalysis' 
username = 'DATALYZER' 
password = 'NMB54321'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
for index, row in df.iterrows():

     cursor.execute("INSERT INTO [DataforAnalysis].[dbo].[Sample_DataML] (Date,betweenDate,Model,Line,Rangeindex,Datum_probe,Max_force,Set_Dim_A,Set_Dim_B,Set_Dim_C,Pivot_Height,Projection,Support,Confidence) values ('"+row.Date+"','"+row.betweenDate+"','"+row.Model+"','"+row.Line+"','"+str(row.Rangeindex)+"','"+str(row.Datum_probe)+"','"+str(row.Max_force)+"','"+str(row.Set_Dim_A)+"','"+str(row.Set_Dim_B)+"','"+str(row.Set_Dim_C)+"','"+str(row.Pivot_Height)+"','"+str(row.Projection)+"','"+str(row.Support)+"','"+str(row.Confidence)+"')")
     # ,(row.Date,row.betweenDate,row.Model,)
                    # ('"+row.Date+"','"+row.betweenDate+"','"+row.Model+"','"+row.Line+"','"+row.Datum_probe+"','"+ a +"','"+row.Set_Dim_A+"','"+row.Set_Dim_B+"','"+row.Set_Dim_C+"','"+row.Pivot_Height+"','"+row.Projection+"','"+row.Support+"','"+row.Confidence+"')")
cnxn.commit()
cursor.close()


